# Chains in LangChain（LangChain中的链）
## Outline大纲
 - LLMChain（大语言模型链）
 - Sequential Chains（顺序链）
    - SimpleSequentialChain
    - SequentialChain
 - Router Chain（路由链）

### 为什么我们需要Chains ？

链允许我们将多个组件组合在一起，以创建一个单一的、连贯的应用程序。链（Chains）通常将一个LLM（大语言模型）与提示结合在一起，使用这个构建块，您还可以将一堆这些构建块组合在一起，对您的文本或其他数据进行一系列操作。例如，我们可以创建一个链，该链接受用户输入，使用提示模板对其进行格式化，然后将格式化的响应传递给LLM。我们可以通过将多个链组合在一起，或者通过将链与其他组件组合在一起来构建更复杂的链。

In [1]:
from dotenv import load_dotenv,find_dotenv
_ = load_dotenv(find_dotenv())

In [2]:
# !pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv("Data.csv")
df.head()

,Product,Review
0,加大床单套装,我订购了一套特大号床单。我唯一的批评是，我希望卖家能提供带有4个枕套的特大号套装。我另外单独...
1,防水手机袋,我喜欢这个防水袋，尽管开口处是硬塑料做的。我不知道它是否会轻易破裂。但我的手机一旦放入袋子里...
2,豪华空气床垫,这款床垫顶部有一个小洞（花了很长时间才找到），而他们提供的补丁没有起作用，可能是因为是在床垫...
3,枕头内胆,这是亚马逊上最好的抱枕填充物。我已经试过好几种，不管你怎么拍打，它们都很便宜且扁平。当你把它...
4,手持奶泡器,我非常喜欢这个产品。但它似乎只能用几个月。公司第一次更换时表现很好（奶泡器从手柄中掉出，无法...


## 1. LLMChain 
LLMChain是一个简单但非常强大的链，也是后面我们将要介绍的许多链的基础。

In [5]:
from langchain_community.chat_models import ChatZhipuAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

初始化语言模型

In [6]:
llm = ChatZhipuAI(temperature=0.9)

初始化prompt，这个prompt将接受一个名为product的变量。该prompt将要求LLM生成一个描述制造该产品的公司的最佳名称

In [8]:
prompt = ChatPromptTemplate.from_template("描述一家生产{product}的公司的最佳名称是什么?")

将llm和prompt组合成链---这个LLM链非常简单，他只是llm和prompt的结合，但是现在，这个链让我们可以以一种顺序的方式去通过prompt运行并且结合到LLM中  
因此，如果我们有一个名为"加大床单套装"的产品，我们可以通过使用chain.invoke将其通过这个链运行  
您也可以输入任何产品描述，然后查看链将输出什么结果

In [9]:
product = "加大床单套装"
(prompt|llm).invoke(product)

AIMessage(content='考虑到该公司专注于生产加大床单套装，并强调舒适、卫生和便捷性，一个最佳的名称可以是“宽舒隔脏专家（GRAREY Comfort Guardian）”。这个名字结合了产品的主要特点：“宽舒”代表了床单的加大加宽设计，提供舒适的睡眠体验；“隔脏专家”则强调了产品在卫生方面的专业性，即使用SMS无纺布材料和真空包装，有效隔开潜在的污染物，保障用户的健康。此外，“GRAREY”这个词如果是公司原有品牌名称，也可以保留以保持品牌识别度。', response_metadata={'token_usage': {'completion_tokens': 115, 'prompt_tokens': 1198, 'total_tokens': 1313}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-c28300aa-3b20-4751-9c8d-e27e9a8ee211-0')

## 2. Sequential Chains

### 2.1 SimpleSequentialChain

顺序链（Sequential Chains）是按预定义顺序执行其链接的链。具体来说，我们将使用简单顺序链（SimpleSequentialChain），这是顺序链的最简单类型，其中每个步骤都有一个输入/输出，一个步骤的输出是下一个步骤的输入

In [10]:
from langchain.chains import SimpleSequentialChain

子链 1

In [12]:
prompt1 = ChatPromptTemplate.from_template("描述一家生产{product}的公司的最佳名称是什么?")
# chain_one = (prompt|llm)
chain_one = LLMChain(llm=llm, prompt=prompt1)

子链 2

In [13]:
prompt2 = ChatPromptTemplate.from_template("写一个20字的描述对于下面这个公司：{company_name}的")
chain_two = LLMChain(llm=llm, prompt=prompt2)

In [14]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two], verbose=True
)

现在我们可以组合两个LLMChain，以便我们可以在一个步骤中创建公司名称和描述  
给一个输入，然后运行上面的链

In [15]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
考虑到该公司专注于生产加大床单套装，并关注卫生、舒适和便捷性，一个最佳的名称可以是“宅旅安心家纺（Zailv Anxin Home Textiles）”。这个名字融合了产品的主要特点：“宅旅”表达了产品适用于外出旅行时也能享受到居家般的安心；“安心”强调了产品为消费者带来的卫生与舒适感；“家纺”则明确了公司的业务范围，即家用纺织品。这样的名称既符合产品特性，又能吸引目标消费者的注意。
"专注加大床单，卫生舒适，宅旅首选。"

> Finished chain.


{'input': '加大床单套装', 'output': '"专注加大床单，卫生舒适，宅旅首选。"'}

### 2.2 SequentialChain

当只有一个输入和一个输出时，简单的顺序链可以顺利完成。但是当有多个输入或多个输出时该如何实现呢？  

我们可以使用普通的顺序链来实现这一点

In [16]:
from langchain.chains import SequentialChain
from langchain_community.chat_models import ChatZhipuAI  #导入OpenAI模型
from langchain.prompts import ChatPromptTemplate          #导入聊天提示模板
from langchain.chains import LLMChain                     #导入LLM链。

初始化语言模型

In [17]:
llm = ChatZhipuAI(temperature=0.9)

接下来我们将创建一系列的链，然后一个接一个使用他们

In [18]:
# prompt模板 1: 翻译成中文（把下面的review翻译成中文）
prompt1 = ChatPromptTemplate.from_template("用中文翻译下面的评论："
                                           "\n\n{Review}")
# chain 1: 输入：Review 输出： 中文的 Review
chain1 = LLMChain(llm=llm,prompt=prompt1,output_key="ch_Review")

In [19]:
# prompt模板 2: 用一句话总结下面的 review
prompt2 = ChatPromptTemplate.from_template("能否用一句话概括以下评论的内容:"
                                           "\n\n{ch_Review}")
# chain 2: 输入：中文的Review   输出：总结
chain2 = LLMChain(llm=llm,prompt=prompt2,output_key="summary")

In [20]:
# prompt模板 3: 下面review使用的什么语言
prompt3 = ChatPromptTemplate.from_template("下面review使用的什么语言,只输出语种:\n\n{Review}")
# chain 3: 输入：Review  输出：语言
chain3 = LLMChain(llm=llm,prompt=prompt3,output_key="language")

In [21]:
# prompt模板 4: 使用特定的语言对下面的总结写一个后续回复
prompt4 = ChatPromptTemplate.from_template(
    "使用下面的指定语言对下面的总结写后续回复:"
    "\n\n总结:{summary}"
    "\n\n语言:{language}"
)
# chain 4: 输入： 总结, 语言    输出： 后续回复
chain4 = LLMChain(llm=llm,prompt=prompt4,output_key="followup_message")

In [22]:
# 对四个子链进行组合

#输入：review    输出：英文review，总结，后续回复 
overall_chain = SequentialChain(
    chains=[chain1,chain2,chain3,chain4],
    input_variables=["Review"],
    output_variables=["ch_Review","summary","language","followup_message"],
    verbose=False
)

让我们选择一篇评论并通过整个链传递它，可以发现，原始review是英文，可以把中文review看做是一种翻译，接下来是根据中文review得到的总结，最后输出的是用英文原文进行的续写信息。

In [23]:
df = pd.read_csv("Data.csv")
# df.head()
review = df.Review[5]

In [24]:
overall_chain.invoke(review)

{'Review': "I find the taste mediocre. The foam doesn't last, which is strange. The ones I buy in-store of the same type taste much better... Is it an old batch or a counterfeit!?!",
 'ch_Review': '我觉得这个味道平平无奇。泡沫也不持久，很奇怪。我店里买的同款味道要好很多……这是不是一个旧批次或者是假货！？！',
 'summary': '"评论者对购买的产品味道和泡沫表现失望，怀疑可能是旧批次或假货。"',
 'language': 'English',
 'followup_message': "Follow-up Reply:\n\nI understand the commenter's disappointment regarding the taste and foam performance of the purchased product. It's concerning that they suspect it might be an old batch or a counterfeit item. We take these concerns seriously and would like to assure our customers that we are looking into the matter immediately. Quality control is a top priority for us, and we will do our best to address any issues that may have arisen. We appreciate the feedback and encourage anyone with similar experiences to reach out to us for further assistance."}

## 3.Router Chain

到目前为止，我们已经学习了LLM链和顺序链。但是，如果您想做一些更复杂的事情怎么办？

一个相当常见但基本的操作是根据输入将其路由到一条链，具体取决于该输入到底是什么。如果你有多个子链，每个子链都专门用于特定类型的输入，那么可以组成一个路由链，它首先决定将它传递给哪个子链，然后将它传递给那个链。

路由器由两个组件组成：

 - 路由器链本身（负责选择要调用的下一个链）
 - destination_chains：路由器链可以路由到的链  

举一个具体的例子，让我们看一下我们在不同类型的链之间路由的地方，我们在这里有不同的prompt:

In [25]:
#第一个提示适合回答物理问题
physics_template = """你是一位非常聪明的物理学教授。\
你在以简洁且易于理解的方式回答物理学问题上非常出色。\
当遇到不知道问题的答案时，你会坦诚表示自己不知道。

这里有一个问题：
{input}"""


#第二个提示适合回答数学问题
math_template = """你是一个非常优秀的数学家。\
你非常擅长解答数学问题。\
你之所以如此出色，是因为你能将难题分解成各个组成部分，\
分别解答这些组成部分，然后再将它们组合起来，\
从而解答更广泛的问题。

这里有一个问题：
{input}"""


#第三个适合回答历史问题
history_template = """你是一位非常出色的历史学家。\
你对横跨多个历史时期的人物、事件和背景拥有卓越的知识和深刻的理解。\
你具备思考、反思、辩论、讨论及评估过去的能力。\
你尊重历史证据，并能利用这些证据来支持你的解释和判断。

这里有一个问题：
{input}"""


#第四个适合回答计算机问题
computerscience_template = """ 你是一位成功的计算机科学家。\
你对创新、合作、前瞻思维、自信、强大的解决问题能力、\
理论与算法的理解以及出色的沟通技巧充满热情。\
你非常擅长回答编程问题。\
你之所以如此出色，是因为你知道如何通过描述命令式步骤来解决问题，\
使机器能轻松解读，并且你知道如何选择一个在时间复杂度和空间复杂度之间\
具有良好平衡的解决方案。

这里有一个问题：
{input}"""

首先需要定义这些提示模板，在我们拥有了这些提示模板后，可以为每个模板命名，然后提供描述。例如，第一个物理学的描述适合回答关于物理学的问题，这些信息将传递给路由链，然后由路由链决定何时使用此子链。

In [26]:
prompt_infos = [
    {
        "name": "物理", 
        "description": "适合回答物理问题", 
        "prompt_template": physics_template
    },
    {
        "name": "数学", 
        "description": "适合回答数学问题", 
        "prompt_template": math_template
    },
    {
        "name": "历史", 
        "description": "适合回答历史问题", 
        "prompt_template": history_template
    },
    {
        "name": "计算机科学", 
        "description": "适合回答计算机科学问题", 
        "prompt_template": computerscience_template
    }
]

导入相关的包

In [27]:
from langchain.chains.router import MultiPromptChain # 导入多提示连
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate,ChatPromptTemplate
from langchain_community.chat_models import ChatZhipuAI  #导入OpenAI模型

定义语言模型

In [28]:
llm = ChatZhipuAI(temperature=0.0)

### LLMRouterChain（此链使用 LLM 来确定如何路由事物）

在这里，我们需要一个多提示链。这是一种特定类型的链，用于在多个不同的提示模板之间进行路由。 但是，这只是你可以路由的一种类型。你也可以在任何类型的链之间进行路由。  

这里我们要实现的几个类是LLM路由器链。这个类本身使用语言模型来在不同的子链之间进行路由。 这就是上面提供的描述和名称将被使用的地方。  

#### 创建目标链  
目标链是由路由链调用的链，每个目标链都是一个语言模型链  

In [29]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm,prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}:{p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
destinations_str

'物理:适合回答物理问题\n数学:适合回答数学问题\n历史:适合回答历史问题\n计算机科学:适合回答计算机科学问题'

#### 创建默认目标链
除了目标链之外，我们还需要一个默认目标链。这是一个当路由器无法决定使用哪个子链时调用的链。在上面的示例中，当输入问题与物理、数学、历史或计算机科学无关时，可能会调用它。

In [30]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm,prompt=default_prompt)

#### 创建LLM用于在不同链之间进行路由的模板 

这包括要完成的任务的说明以及输出应该采用的特定格式。

In [31]:
# from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
MULTI_PROMPT_ROUTER_TEMPLATE = """
Given a raw text input to a language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a description of what the prompt is best suited for.\
You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT".
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the response) >>
<< OUTPUT (must end with ```) >>
"""

#### 构建路由链
首先，我们通过格式化上面定义的目标创建完整的路由器模板。这个模板可以适用许多不同类型的目标。 因此，在这里，您可以添加一个不同的学科，如英语或拉丁语，而不仅仅是物理、数学、历史和计算机科学。  

接下来，我们从这个模板创建提示模板  

最后，通过传入llm和整个路由提示来创建路由链。需要注意的是这里有路由输出解析，这很重要，因为它将帮助这个链路决定在哪些子链路之间进行路由。

In [32]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
print(router_template)


Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for.You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.
<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT".
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理:适合回答物理问题
数学:适合回答数学问题
历史:适合回答历史问题
计算机科学:适合回答计算机科学问题

<< INPUT >>
{input}

<< OUTPUT (must include ```json at the start of the response) >>
<< OUTPUT (must end with `

In [33]:
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)
router_chain = LLMRouterChain.from_llm(llm,router_prompt)

最后，将所有内容整合在一起，创建整体链路

In [34]:
#多提示链
chain = MultiPromptChain(
    router_chain = router_chain,              #路由链路
    destination_chains = destination_chains,  #目标链路
    default_chain = default_chain,            #默认链路
    verbose = True
)

#### 进行提问

如果我们问一个物理问题，我们希望看到他被路由到物理链路

In [35]:
# 问题：什么是黑体辐射？
res = chain.invoke("什么是黑体辐射?")
print(res)



> Entering new MultiPromptChain chain...
物理: {'input': '什么是黑体辐射?'}
> Finished chain.
{'input': '什么是黑体辐射?', 'text': '黑体辐射是指理想化的物体，它能够吸收所有入射的电磁辐射，不反射也不透过，而且在任何温度下都会以一定的频谱分布发射电磁辐射。在热平衡状态下，这个频谱分布只依赖于黑体的温度，与黑体的材质和形状无关。\n\n简而言之，可以想象一个完美的“黑色”容器，无论里面是什么温度，它都能完美地吸收所有光和其他电磁波，并且根据其温度以特定的方式发射出电磁波。这种辐射的强度和频谱分布可以用普朗克黑体辐射定律来描述，它是量子物理学的基础之一。\n\n在实际应用中，虽然完美的黑体并不存在，但许多物体在特定波长范围内的行为接近于黑体，因此黑体辐射的概念在热辐射、天体物理学、光谱学和许多其他领域都有着重要的应用。'}


In [36]:
print(res["text"])

黑体辐射是指理想化的物体，它能够吸收所有入射的电磁辐射，不反射也不透过，而且在任何温度下都会以一定的频谱分布发射电磁辐射。在热平衡状态下，这个频谱分布只依赖于黑体的温度，与黑体的材质和形状无关。

简而言之，可以想象一个完美的“黑色”容器，无论里面是什么温度，它都能完美地吸收所有光和其他电磁波，并且根据其温度以特定的方式发射出电磁波。这种辐射的强度和频谱分布可以用普朗克黑体辐射定律来描述，它是量子物理学的基础之一。

在实际应用中，虽然完美的黑体并不存在，但许多物体在特定波长范围内的行为接近于黑体，因此黑体辐射的概念在热辐射、天体物理学、光谱学和许多其他领域都有着重要的应用。


In [37]:
chain.invoke("有三进制吗?")



> Entering new MultiPromptChain chain...
数学: {'input': '三进制数是什么，它如何工作？'}
> Finished chain.


{'input': '三进制数是什么，它如何工作？',
 'text': '三进制数是一种计数系统，和我们所熟悉的十进制数一样，但它基于3而不是10。在三进制系统中，每个位置的数值可以是从0到2的任意整数，因为三进制只有三个数字：0、1和2。这与十进制系统中的0到9相对应。\n\n三进制数是这样工作的：\n\n1. **位权（Place Value）**：在三进制中，每个位置代表3的幂次。最右边的位置是3的0次方（即1），往左每增加一个位置，幂次就增加1。例如：\n   - \\(2 \\times 3^0 = 2 \\times 1 = 2\\)\n   - \\(1 \\times 3^1 = 1 \\times 3 = 3\\)\n   - \\(0 \\times 3^2 = 0 \\times 9 = 0\\)\n   - \\(1 \\times 3^3 = 1 \\times 27 = 27\\)\n\n2. **数值表示**：因此，三进制数“102”表示为：\n   \\(1 \\times 3^2 + 0 \\times 3^1 + 2 \\times 3^0 = 9 + 0 + 2 = 11\\)（在十进制中）\n\n3. **进位**：当三进制中的一个位置上的数值达到3时，就需要进位，就像十进制中从9到10一样。例如：\n   - \\(2 + 1 = 3\\)，在三进制中，这会写成“10”（即十进制中的3），因为3是3的1次方。\n\n4. **转换**：将三进制数转换为十进制，可以按照每个位置的位权乘以相应的数字，然后将它们相加。\n\n5. **运算**：在三进制下，加法、减法、乘法和除法等基本运算与十进制类似，但需要遵循三进制的规则。\n\n下面是一个三进制加法的例子：\n\n```\n  1 2 1  (三进制)\n+ 2 1 0  (三进制)\n---------\n  1 1 2 1 (三进制)\n```\n\n在这个例子中，最右边的位置相加得到1（0+1），然后是2（1+1），接着是1（1+0），最后是进位得到1（因为3等于三进制的“10”）。\n\n三进制数在计算机科学和数学的某些领域有特殊用途，例如在某些类型的编码和密码学中。它们也可以用于理解数字系统的基础概念。'}

In [38]:
chain.invoke("什么是显卡?")



> Entering new MultiPromptChain chain...
计算机科学: {'input': '什么是显卡?'}
> Finished chain.


{'input': '什么是显卡?',
 'text': '显卡，也称为图形处理单元（Graphics Processing Unit，GPU），是一种专门设计用来快速处理和渲染图像的电子组件。它是计算机系统中的一个重要部分，负责将计算机生成的图像数据转换成显示器可以理解的格式，并在屏幕上显示出来。\n\n显卡的主要功能包括：\n\n1. 图形渲染：为计算机游戏、图形设计、视频编辑等应用提供实时的图像渲染能力。\n2. 视频输出：将处理过的图像数据输出到显示器、电视或其他视觉输出设备。\n3. 计算加速：在现代显卡中，GPU还可以用于通用计算任务，尤其是那些需要大量并行处理的任务，如机器学习、科学模拟等。\n\n显卡的关键特性包括：\n\n- 处理能力：由核心数量、核心频率和架构决定。\n- 内存：显卡上的专用内存（如GDDR5或GDDR6），用于存储正在处理的图像数据和相关的计算数据。\n- 带宽：影响显卡处理和传输数据的能力。\n- 兼容性：与计算机系统的接口标准，如PCI Express。\n- 驱动程序：软件组件，允许操作系统和应用程序与显卡硬件通信。\n\n在解决编程问题时，显卡的并行处理能力可以被用来加速某些算法的执行，尤其是那些可以映射到图形处理流水线的算法。例如，在处理复杂图形渲染或大规模并行计算任务时，合适的显卡和相应的编程技术可以大幅提高效率，找到时间复杂度和空间复杂度之间的良好平衡。'}

In [39]:
# 问题：为什么我们身体里的每个细胞都包含DNA？
chain.invoke("为什么我们体内的每个细胞都含有DNA?")



> Entering new MultiPromptChain chain...
生物: {'input': '为什么我们体内的每个细胞都含有DNA？'}

ValueError: Received invalid destination chain name '生物'

In [ ]:
# from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
# print(MULTI_PROMPT_ROUTER_TEMPLATE)

In [40]:
chain.invoke("你知道李白是谁嘛?")



> Entering new MultiPromptChain chain...
历史: {'input': '你知道李白是谁嘛?'}
> Finished chain.


{'input': '你知道李白是谁嘛?',
 'text': '当然，李白是唐代（公元618-907年）著名的诗人，被誉为“诗仙”。他的诗歌以其豪放、奔放、想象丰富和语言的鲜明特色而著称，是唐诗中的佼佼者。\n\n李白出生于701年，据传是唐朝的皇室远亲，但他的一生并未在朝中担任要职。他的生平充满了传奇色彩，据说他遍历名山大川，酷爱杯中物，追求自由与超脱。他的诗作中，既有歌颂自然美景的，也有表达对英雄理想和远大抱负的追求，还有反映个人失意和孤独的。\n\n李白的诗歌在中国文学史上有着极其重要的地位，对后世文人墨客产生了深远的影响。他的部分代表作如《将进酒》、《庐山谣》、《夜泊牛渚怀古》等，至今仍广为传诵。通过他的诗歌，我们可以窥见唐代的社会风貌、文化特色以及当时文人的思想情感。'}